In [17]:
import xlrd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import re
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from operator import itemgetter
from matplotlib import pyplot as plt
import numpy as np
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_file
import nltk
import gensim
import string
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [18]:
# Open the workbook
xl_workbook = xlrd.open_workbook("VR_AR_DATASET_USPTO.xlsx") 

# Grabbing the Required Sheet by index 
xl_sheet = xl_workbook.sheet_by_index(0)

#Initialising the Lists Required

# Open the workbook
xl_workbook2 = xlrd.open_workbook("stopwords.xlsx") 

# Grabbing the Required Sheet by index 
xl_sheet2 = xl_workbook2.sheet_by_index(0)

#Initialising the Lists Required

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

lemmatiser = WordNetLemmatizer()


output_file("test.html")

stemmer = nltk.SnowballStemmer("english")

topics = []
currTopicList = []
overallTopicList = []
input_data = []




def tokenize(text):
    lemmatized_words = []
    text = text.lower()
    text = text.strip("\n")
    text = re.sub(r'\\n',' ',text)
    text = re.sub(r'[^\w\s]',' ',text)
    tokens = nltk.word_tokenize(text)
    tokens_pos = pos_tag(tokens)
    count = 0
    for token in tokens:
        pos = tokens_pos[count]
        pos = get_wordnet_pos(pos[1])
        if pos != '':
            lemma = lemmatiser.lemmatize(token, pos)
        else:
            lemma = lemmatiser.lemmatize(token)
        lemmatized_words.append(lemma)
        count+=1
    return lemmatized_words


num_cols = xl_sheet.ncols   #Number of columns

stoplist = stopwords.words('english')

In [19]:
for row_idx in range(0, xl_sheet2.nrows): 
    col_idx = 0
    cccc = str(xl_sheet2.cell(row_idx, col_idx))
    bc = cccc.decode('utf-8')
    stoplist.append(bc)

for row_idx in range(2, xl_sheet.nrows): 
#for row_idx in range(2, 10):   # Iterate through rows

    # for col_idx in range(1, 2):  
    #     cell_obj = xl_sheet.cell(row_idx, col_idx)  # Get cell object by row, col
    #     var1 = str(cell_obj)[7:-1]
 
    col_idx = 1
    doc1 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 2
    doc2 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 3
    doc3 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 4
    doc4 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx == 5
    doc5 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx == 6
    doc6 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 20
    doc7 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 38
    doc8 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx =65
    doc9 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 69
    doc10 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx =72
    doc11 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx =74
    doc12 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 78
    doc13 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx =79
    doc14 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx =81
    doc15 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    col_idx = 82
    doc16 = str(xl_sheet.cell(row_idx, col_idx))[7:-1]
    doc1 = doc1 + doc2 + doc3 + doc4 + doc5 + doc6 + doc7 + doc8 + doc9 + doc10+ doc11 + doc12 + doc13 + doc14 + doc15 + doc16

    doc_complete = [doc1]


    stop = set(stoplist)
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()

    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized

    doc_clean = [clean(doc).split() for doc in doc_complete]       


    # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary(doc_clean)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=1, id2word = dictionary, passes=10)

    #Results
    #curr_topic= str(ldamodel.print_topics(num_topics=3, num_words=3)[0])[12:-2]
    curr_topic= str(ldamodel.print_topics(num_topics=1, num_words=5)[0])

    #print(curr_topic)



    
    currTopicList =re.findall(r'"([^"]*)"', curr_topic)
    N = len(currTopicList)


    for i in range(0, N): 
        overallTopicList.append(currTopicList[i])

    topics.append(currTopicList)
    #print currTopicList
    #abc = currTopicList[0]+' '+currTopicList[1]+' '+currTopicList[2]+' '+currTopicList[3]+' '+currTopicList[4]+' '+currTopicList[5]+' '+currTopicList[6]+' '+currTopicList[7]+' '+currTopicList[8]+' '+currTopicList[9]
    abc = currTopicList[0]+' '+currTopicList[1]+' '+currTopicList[2]+' '+currTopicList[3]+' '+currTopicList[4]
    
    abc = abc.strip("\n")
    input_data.append(abc)


In [20]:
dictTopicList = {x:overallTopicList.count(x) for x in overallTopicList}
dictTopicList = {k: v for k, v in dictTopicList.iteritems() if v >3}

#print dictTopicList
import operator
sorted_x = sorted(dictTopicList.items(), key=operator.itemgetter(1),reverse=True)
i = 0
freq=[]
terms=[]
while ( i < len(sorted_x)):
    freq.append(float(sorted_x[i][1]))
    terms.append(sorted_x[i][0])
    # print (type(sorted_x[i][0]))
    # print (type(sorted_x[i][1]))
    i = i +1

In [21]:

#Plotting the Graph
N = len(sorted_x)
ind = np.arange(N)      #The x locations for the groups
width = 0.35            #The width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, freq, width, color='r')
#rects2 = ax.bar(ind + width, patent_citation_count, width, color='y')

# #Add some text for labels, title and axes ticks

ax.set_ylabel('Frequency')
ax.set_title('Number of times occured')
ax.set_xticks(ind)
ax.set_xticklabels(terms, rotation='vertical')
abc = "Total number of topics =" + str(N)
abc2 = "Total number of patent analysed =" + str(xl_sheet.nrows)

#Average no of topics per patent analysed

i = 0
abc3 = 0
while (i < N):
	abc3 = abc3 + freq[i]
	i = i +1 

abc5= abc3/xl_sheet.nrows

h = round(abc5, 2)
abc4 = "Average no of topics per patent =" + str(h) 


ax.annotate(abc, xy=(0.3,0.9),xycoords='axes fraction',
             fontsize=14)
ax.annotate(abc2, xy=(0.3,0.8),xycoords='axes fraction',
             fontsize=14)
ax.annotate(abc4, xy=(0.3,0.7),xycoords='axes fraction',
              fontsize=14)


#Attach a text label above each bar displaying its height
def autolabel(rects):    
    for rect in rects:
        height = rect.get_height()
        if height != 0 :
            ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,'%d' % int(height),ha='center', va='bottom')
autolabel(rects1)


# Tweak spacing to prevent clipping of tick-labels
plt.subplots_adjust(bottom=0.15)
axes = plt.gca()
axes.set_ylim([0,np.max(freq)+10])


#plt.savefig('histogram.png', dpi = 1000)
plt.show()



In [22]:
user_ids = []
subreddit_ids = []
subreddit_to_id = {}
j= 0

for i in input_data:
	
    for sr in i.rstrip().split(" ")[0:]: 
        if sr not in subreddit_to_id: 
            subreddit_to_id[sr] = len(subreddit_to_id)
        user_ids.append(j)
        subreddit_ids.append(subreddit_to_id[sr])
    j+=1

In [23]:
%%time


import numpy as np
from scipy.sparse import csr_matrix 

rows = np.array(subreddit_ids)
cols = np.array(user_ids)
data = np.ones((len(user_ids),))
num_rows = len(subreddit_to_id)
num_cols = j

# the code above exists to feed this call
adj = csr_matrix( (data,(rows,cols)), shape=(num_rows, num_cols) )
print adj.shape
print ""

# now we have our matrix, so let's gather up a bit of info about it
users_per_subreddit = adj.sum(axis=1).A1
subreddits = range(len(subreddit_to_id))
for sr in subreddit_to_id:
    subreddits[subreddit_to_id[sr]] = sr
subreddits = np.array(subreddits)

(925, 763)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.89 ms


In [24]:
new=[]
total=[]
patent=[]
total_patent=[]

for i in input_data:
    new= i.rstrip().split(" ")[0:]
    total.append(new)

In [25]:
patents = range(len(subreddit_to_id))
for sr in subreddit_to_id:

    z =0
    for j in total:
        for w in total[z]: 
            if w == sr:
            	patent.append(z)
        z = z +1 
    patents[subreddit_to_id[sr]] = patent
    #total_patent.append(patent)
    patent = []
patents = np.array(patents)

In [26]:
%%time

from sklearn.decomposition import TruncatedSVD 
from sklearn.preprocessing import normalize 

svd = TruncatedSVD(n_components=30)
embedded_coords = normalize(svd.fit_transform(adj), norm='l1')
print embedded_coords.shape

(925, 30)
CPU times: user 64 ms, sys: 256 ms, total: 320 ms
Wall time: 112 ms


In [27]:
import bokeh.plotting as bp
from bokeh.plotting import figure
from bokeh.models import HoverTool 
bp.output_notebook()
row_selector = np.where(users_per_subreddit>1)

Loading BokehJS ...

In [28]:
%%time

from scipy.stats import rankdata
embedded_ranks = np.array([rankdata(c) for c in embedded_coords.T]).T

from sklearn.cluster import KMeans
n_clusters = 20
km = KMeans(n_clusters)
clusters = km.fit_predict(embedded_ranks)

CPU times: user 364 ms, sys: 992 ms, total: 1.36 s
Wall time: 521 ms


In [29]:
from bokeh.models import HoverTool

colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c", 
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5", 
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f", 
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])

In [30]:
%%time

from sklearn.manifold import TSNE
xycoords = TSNE().fit_transform(embedded_coords[row_selector])

CPU times: user 3.4 s, sys: 736 ms, total: 4.14 s
Wall time: 3.95 s


In [31]:
source =bp.ColumnDataSource({"subreddit": subreddits[row_selector],
                           "patents": patents[row_selector]}
                           )


hover = HoverTool(
tooltips="""
        <div>
            <div>
            <span style="font-size: 10px; font-weight: bold;">@subreddit</span>
            </div>
            <div>
            <span style="font-size: 5px;font-weight: bold;">@patents</span>
            </div>
        </div>
        """   )

TOOLS= "pan,wheel_zoom,box_zoom,crosshair,reset,previewsave"

tbf = bp.figure(plot_width=900, plot_height=700, title="Subreddit Map by t-SNE",
       tools=[hover,TOOLS],
       x_axis_type=None, y_axis_type=None, min_border=1)
tbf.scatter(
    x = embedded_coords[:,0],
    y = embedded_coords[:,1],
    color= colormap[clusters[row_selector]], 
    radius= np.log2(users_per_subreddit[row_selector])/6000, 
    source=source
)



bp.show(tbf)


/usr/local/lib/python2.7/dist-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python2.7/dist-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/usr/local/lib/python2.7/dist-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh